# Project 4
## Students:
 - Bryson Gullett
 - Robert Schaffer
 - Matthew Dixson
 

In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import optimizers
from tensorflow.keras import layers
import numpy as np
import re
import random as rand

In [58]:
print(tf.__version__)# you may want to upgrade to 2.10.0

2.10.0


### Please Use Markdown
> for markdown, see here: https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

## Task 1

In [59]:
class TransformerModel():
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=80, rate=0.1):
        #initailize variables
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.ff_dim = ff_dim
        self.maxlen = maxlen
        self.rate = rate

    def TransformerBlock(self, inputs):
        #create the transformer block as discribed in the writeup, use the Keras functional API (https://keras.io/guides/functional_api/)
        #MultiHeadAttention layer, specifiy 'use_causal_mask=True' (https://keras.io/api/layers/attention_layers/multi_head_attention/)
        #LayerNormalization layer, specifiy 'epsilon=1e-6' (https://keras.io/api/layers/normalization_layers/layer_normalization/)
        #Use the rate variable for the dropout layers and remember to use two dense layers
        #See assignment and its figures for more details.
        multihead_atten = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.num_heads, dropout=self.rate)(inputs, inputs, use_causal_mask=True)
        add1 = layers.Add()([multihead_atten, inputs])
        layer_norm1 = layers.LayerNormalization(epsilon=1e-6)(add1)
        dense1 = layers.Dense(self.ff_dim, activation='relu')(layer_norm1)
        dense2 = layers.Dense(self.ff_dim, activation='relu')(dense1)
        dropout1 = layers.Dropout(self.rate)(dense2)
        add2 = layers.Add()([dropout1, layer_norm1])
        layer_norm2 = layers.LayerNormalization(epsilon=1e-6)(add2)

        return layer_norm2
    
    def EmbeddingLayer(self, inputs):
        #create the embedding layer
        #create (1) an embedding for the tokens and (2) an embedding for the positions
        #you can use https://keras.io/api/layers/core_layers/embedding/ Embedding class
        #you can use tf.range to enocde positions
        #add (1) and (2) and return the layer
        positions = layers.Lambda(lambda x: tf.multiply(tf.ones_like(x, dtype='int32'), tf.range(self.maxlen)))(inputs)
        token_embeddings = layers.Embedding(self.vocab_size, self.embed_dim, input_length=self.maxlen)(inputs)
        position_embeddings = layers.Embedding(self.maxlen, self.embed_dim, input_length=self.maxlen)(positions)

        add = layers.Add()([token_embeddings, position_embeddings])
        return add
    
    def create_model(self):
        #combine the EmbeddingLayer and num_blocks TransformerBlocks to create the model, use the Keras functional API (https://keras.io/guides/functional_api/)
        #use the SparseCategoricalCrossentropy loss function (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)
        inputs = keras.Input(shape=(self.maxlen,))
        embedding_layer = self.EmbeddingLayer(inputs)
        prev_layer = embedding_layer
        for _ in range(self.num_blocks):
            transformer_block = self.TransformerBlock(prev_layer)
            prev_layer = transformer_block
        final_dense = layers.Dense(self.vocab_size, activation='softmax')(prev_layer)
        model = keras.Model(inputs=inputs, outputs=final_dense)
        self.opt = optimizers.Adam(learning_rate=0.0001)
        self.loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(loss=keras.losses.SparseCategoricalCrossentropy, optimizer=self.opt)
        return model
    
my_model = TransformerModel(1000)
print(my_model.create_model().summary())

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_36 (InputLayer)          [(None, 80)]         0           []                               
                                                                                                  
 lambda_35 (Lambda)             (None, 80)           0           ['input_36[0][0]']               
                                                                                                  
 embedding_70 (Embedding)       (None, 80, 256)      256000      ['input_36[0][0]']               
                                                                                                  
 embedding_71 (Embedding)       (None, 80, 256)      20480       ['lambda_35[0][0]']              
                                                                                           

## Task 2

In [60]:
class DataSet():
    def __init__(self, filename, len):
        #load the text from the file
        self.text = ''
        self.len = len
        with open(filename) as fin:
            for line in fin:
                self.text += line

    def prep_text(self):
        #remove all punctuation, set to lowercase, remove duplicate spaces and other whitespace (keep newlines)
        self.text = re.sub(r'[^\w\s]', ' ', self.text)
        self.text = re.sub(r' +', ' ', self.text)
        self.text = re.sub(r'â', '', self.text)
        
        
        
        
    def tokenize_text(self):
        #seperate into words, create a vocab and convert the text to a list of numbers using the vocab such that each unique word is represented by its own number number
        self.text = self.text.split()
        self.vocab = np.unique(self.text)
        self.vocab = np.append(self.vocab, ['PAD'])
        self.vocab_nums = [i for i in range(len(self.vocab))]
        self.vocab_dict = dict(zip(self.vocab, self.vocab_nums))
        

    def create_dataset(self):
        #split the tokenized data into sequences of length len, return the sequences and vocab
        self.prep_text()
        self.tokenize_text()
        X = []
        Y = []
        for i in range(int(len(self.text)/self.len)+1):
            sequence_X = []
            sequence_Y = []
            for j in range(self.len):
                if i*self.len+j < len(self.text):
                    sequence_X.append(self.vocab_dict[self.text[i*self.len+j]])
                else:
                    sequence_X.append(self.vocab_dict['PAD'])
                if i*self.len+j+1 < len(self.text):
                    #y = np.zeros(len(self.vocab))
                    #y[self.vocab_dict[self.text[i*self.len+j+1]]] = 1
                    #y = [0 if self.vocab_dict[self.text[i*self.len+j+1]] != k else 1 for k in range(len(self.vocab))]
                    sequence_Y.append(self.vocab_dict[self.text[i*self.len+j+1])
                else:
                    #y = np.zeros(len(self.vocab))
                    #y[self.vocab_dict['PAD']] = 1
                    #y = [0 if self.vocab_dict['PAD'] != k else 1 for k in range(len(self.vocab))]
                    sequence_Y.append([self.vocab_dict['PAD']])
            X.append(sequence_X)
            Y.append(sequence_Y)
        return X, Y, self.vocab

## Task 3

In [61]:
class GenerateText():
    def __init__(self, model, vocab, vocab_dict):
        self.model = model
        self.vocab = vocab
        self.vocab_dict = vocab_dict
    
    def generate_text(self, start_string, num_generate=100, seq_len=80):
        #generate text using the model and vocab, start with the start_string and generate num_generate words
        model_input = [self.vocab_dict['PAD'] for _ in range(seq_len)]
        
        #Text is the output of generate_text()
        text = start_string
        #Tokenize string to encode into integers
        start_string = start_string.split()
        #Get starting index for first word generated
        next_index = len(start_string)
        #Generate model input
        for i in range(len(start_string)):
            model_input[i] = self.vocab_dict[start_string[i]]
        
        #Generate text
        for i in range(num_generate):
            prediction = self.model.predict(np.array([model_input]))
            # print(len(prediction))
            # print(len(prediction[0]))
            # print(len(prediction[0][0]))
            word_index = np.argmax(prediction[0][next_index])
            next_index += 1
            if word_index >= len(self.vocab):
                print('Invalid word index: ', word_index, "Valid max word index = ", len(self.vocab))
                word_index = self.vocab_dict['PAD']
            if next_index < seq_len:
                model_input[next_index] = word_index
            else:
                model_input.pop(0)
                model_input.append(word_index)
                next_index -= 1
          
            next_word = self.vocab[word_index]
            text += ' ' + next_word
            # print(model_input)
        return text
    
    def generate_random_text(self, num_generate=100):
        text = ''
        for _ in range(num_generate):
            word = self.vocab[rand.randrange(len(self.vocab))]
            text += word + ' '
            if word == 'PAD':
                break
        return text   

In [62]:
data = DataSet('beatles.txt', 80)
_, _, vocab = data.create_dataset()
transformer = TransformerModel(len(vocab))
model = transformer.create_model()
text = GenerateText(model, vocab, data.vocab_dict)
out = text.generate_text("i read a crowd of people", num_generate=5)
print(out)
out = text.generate_random_text(num_generate=5)
print(out)
# print(len(out.split()))

1/1 [==============================] - 0s 48ms/step
i read a crowd of people poop think hai al penny
beautiful ah all dime being 


## Task 4: Model Traning and Testing

In [95]:
#Train the model while periodically generating text to show progress
def train_model(model, vocab, X, Y, epochs=50):
    # https://keras.io/guides/writing_a_training_loop_from_scratch/
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    for _ in range(epochs):
        
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(X, training=True).numpy()  # Logits for this minibatch
            # print(logits)
            # Compute the loss value for this minibatch.
            print(Y.shape)
            print(logits.shape)
            loss_value = loss_fn(Y.reshape(1,452, 80, 2506), logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        print(loss_value)
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        model.optimizer.apply_gradients(zip(grads, model.trainable_weights))



        # model.fit(x,y)
        # text = GenerateText(model, vocab, data.vocab_dict)
        # out = text.generate_random_text(num_generate=10)
        # print(out)

    return model

In [97]:
data = DataSet('beatles.txt', 80)
x, y, vocab = data.create_dataset()
transformer = TransformerModel(len(vocab), num_blocks=3)
model = transformer.create_model()

X = []
for e in x:
    X.append(np.asarray(e).astype(np.int64))
X = np.asarray(X)

Y= []
for i in y:
    Y.append(np.asarray(i).astype(np.int64))
    # print(Y[0].shape)
Y = np.asarray(Y)

train_model(model, vocab, X, Y)
# text = GenerateText(model, vocab, data.vocab_dict)
# out = text.generate_text("i read a crowd of people", num_generate=5)
# print(out)

(452, 80, 2506)
(452, 80, 2506)


ValueError: `labels.shape` must equal `logits.shape` except for the last dimension. Received: labels.shape=(90616960,) and logits.shape=(36160, 2506)


# Report

## Introduction

## Results

## Conclusion

## How to Run Code

Please include any special libraries and list your tf version here.